In [1]:
%pip install sktime==0.11.3


  Using cached sktime-0.11.3-py3-none-any.whl (6.7 MB)


  Attempting uninstall: sktime
    Found existing installation: sktime 0.11.2
    Uninstalling sktime-0.11.2:
      Successfully uninstalled sktime-0.11.2


In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import tensorflow as tf
from pickle import dump

from sktime.classification.distance_based import ProximityForest
from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
print(np.shape(X))
print(type(X))
precision = []
accuracy = []
f1 = []
recall = []
def gpu(X, y, kf):
  with tf.device('/device:GPU:0'):
    i= 0
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)    
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(X_train.head())
        #print(type(X_train))
        print("Treining PF...")
        PF =     ProximityForest(n_estimators=50)
        print(f"Interaction {i}")
        print("In trainning...")
        PF.fit(X_train, y_train)
        print("In prediction...")
        i=i+1
        
        # Predict and print accuracy
        predictions = PF.predict(X_test)
        precision.append(precision_score(y_test, predictions, average='macro',zero_division=1))
        accuracy.append(accuracy_score(y_test, predictions))
        recall.append(recall_score(y_test, predictions, average='macro',zero_division=1))
        f1.append(f1_score(y_test, predictions, average='macro'))
        #print("In prediction...")
        # Predict and print accuracy
       # predictions = HC2.predict(X_test)
    print(type(X_train))
    print(np.shape(X_train))
    print(f"Precision: {np.mean(precision)}, Accuracy:{np.mean(accuracy)}, Recall: {np.mean(recall)}, F1: {np.mean(f1)}")
    return np.mean(precision), np.mean(accuracy), np.mean(recall), np.mean(f1), np.std(accuracy), PF

(12256, 1)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("Using MirroredStrategy")
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  print("Using Default Strategy")
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))
  precision, accuracy, recall, f1, std, PF_model = gpu(X,y,kf)

Using MirroredStrategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Treining PF...
Interaction 0
In trainning...


In [ ]:
df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','std'])
df_metrics['precision'] = precision
df_metrics['accuracy'] = accuracy
df_metrics['recall'] = recall
df_metrics['f1_score'] = f1
df_metrics['std'] = std
df_metrics

In [ ]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv('results/'+ 'df_metrics_PF.csv', index=False)
#save the model
print("Saving model...")
dump(PF_model, open('results/'+'PF_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

In [ ]:
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))